# Data Wrangling Project with Python and MongoDB
## 1.What is Data Wrangling?
## 2.Project Explanation
## 3.Data Exploration
## 4.Conclusion
<hr/>

## 1.What is data wrangling?
#### Definition: 
The process of manually converting or mapping data from one "raw" form into another format that allows for more convenient consumption of the data with the help of semi-automated tools.
#### Why do we need it?
After having the data, usually the data's shape is not consistant and clean. So, we need to reshape the data and clean unnecessary things. Espisally when we take data using web scrapping methods and when combining multiple sources of data together. 
#### Tools used:
To clean (wrangle) data, you can do in one of many ways. The listed tools below are most used in these days.
<ol>
    <li>Manually by hand [Not good way with huge data].</li>
    <li>Programming [Scripts to complete a spesific task, excellent with most cases].</li>
    <li>Applications [Commerical or open source applications, most of them works with just a small list of data types].</li>
</ol>


## 2.Project Explanation:
###### The project aims to help the data analyst understanding the wrangling process, from getting data cleaning it and then store it on local database. 
This project is about the Open Street Map, which is a collaborative Geographical Information System (GIS). Open Street Map allows the community adding, updating and deleting data from maps. Open Street Map (OSM) lunched by Steve Coast 12 years ago and it has more than 3 milion users. 

###### How project works?
To complate the project you need to download an OSM XML dataset for one of areas you are intrested in. Then audit the data in many ways.
<ul>
    <li>Languages</li>
    <li>Data formats</li>
    <li>Abbreviation</li>
    <li>Etc...</li>
</ul>

Also, statistical overview of the dataset must be provided for:
<ul>
    <li>Size of the file</li>
    <li>Number of unique users</li>
    <li>Number of nodes and ways</li>
    <li>Number of chosen type of nodes, like cafes, shops etc.</li>
</ul>

Finally, you need to provide suggestions for improving, analyzing the data and includes thoughtful discussion about the benefits as well as some anticipated problems in implementing the improvement.
<hr/>

### 3. Data Exploration
I selected Cape Town, South Africa as my area to be cleaned. I will do the following steps:
<ol>
    <li>Download map's data</li>
    <li>Prepare workspace</li>
    <li>Select dataset file</li>
    <li>Check file size</li>
    <li>Calculate total tags</li>
    <li>Calculate total users</li>
    <li>Audit abbreviations</li>
    <li>Change abbreviations</li>
    <li>Convert data JSON</li>
    <li>Storing data in MongoDB</li>
    <li>Statistical overview</li>
</ol>

#### 1.Download Map's Data
I Downloaded Cape Town from the https://mapzen.com/data/metro-extracts datasets. The dataset path is https://s3.amazonaws.com/metro-extracts.mapzen.com/cape-town_south-africa.osm.bz2
<hr/>

#### 2.Prepare Workspace
Here, I will import the libraries which I will use in the cleaning process. 

In [1]:
import xml.etree.cElementTree as ET
import pprint
import re
import os
import codecs
import json
import re
from collections import defaultdict
from pymongo import MongoClient


#### 3.Select Dataset File
The selected file will be used in the clearnce process.

In [2]:
OSM_FILE = 'cape-town_south-africa.osm'

#### 4.Check File Size
Here we will check the file's size and print it out.

In [3]:
# this function takes a number of file size in bytes and convert it to other sizes
def convert_bytes_to_size(num):
    for x in ['Bytes', 'KB', 'MB', 'GB', 'TB']:
        if num < 1024.0:
            return "%3.1f %s" % (num, x)
        num = num / 1024.0

def get_file_size(file_path):
    if os.path.isfile(file_path):
        file_info = os.stat(file_path)
        return convert_bytes_to_size(file_info.st_size)
    
size = get_file_size(OSM_FILE)
print ('File size', size)


('File size', '283.0 MB')


The size of the dataset is 283 MB. Whihe means it is bigger than 50 MB.


<hr/>

#### 5.Calculate Total Tags
Here we will loop over the dataset file and count the number of occerence of each tag [type]. This will help us understanding dataset and its structure.

Then, we will count tags based on the valdiaty and style.

In [4]:
# Useing Element Tree, we will loop over the dataset and count the number of appercence of tags.
def get_tags_count(filename):
        tags_dictionary = {}
        for event, element in ET.iterparse(filename):
            if element.tag in tags_dictionary: 
                tags_dictionary[element.tag] = tags_dictionary[element.tag] + 1
            else:
                tags_dictionary[element.tag] = 1
        return tags_dictionary

pprint.pprint(get_tags_count(OSM_FILE))

{'bounds': 1,
 'member': 30724,
 'nd': 1554908,
 'node': 1350871,
 'osm': 1,
 'relation': 3033,
 'tag': 604348,
 'way': 212159}


The dataset set contains the above numbers of tags. Which means it is little bit big.

In [5]:
'''
    reguler expressions where we will check all tags...
    "lower", valid tags in lowercase,
    "lower_colon", valid tags with a colon in their names,
    "problemchars", tags with problematic characters, and
    "other",other tags that do not fall into the other three categories.
'''
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
        for tag in element.iter('tag'):
            k = tag.get('k')
            if lower.search(k):
                keys['lower'] += 1
            elif lower_colon.search(k):
                keys['lower_colon'] += 1
            elif problemchars.search(k):
                keys['problemchars'] += 1
            else:
                keys['other'] += 1
    return keys


def check_key_types(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for event, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

keys = check_key_types(OSM_FILE)
pprint.pprint(keys)

{'lower': 555062, 'lower_colon': 48113, 'other': 1166, 'problemchars': 7}


Most of dataset tags are valid with lowercase style. Where 7 tags are not valid.

<hr/>

#### 6.Calculate Total Users
Now, we will check all edits and who edited them. A bigger number means more comunity, where a smaller number means usually using automated bots.

In [6]:
# Check all elemetn and who edit them. We are using set to count users only once [not allowing duplication].
def get_users_set(filename):
    users_set = set()
    for event, element in ET.iterparse(filename):
        for item in element:
            if 'uid' in item.attrib:
                users_set.add(item.attrib['uid'])
    return users_set
users = get_users_set(OSM_FILE)
len(users)


1538

Our dataset puplated and edited by 1538 users.

<hr/>

#### 7.Audit Abbreviations
Here we will list all appriviations and match them with current appriviations in the dataset.

In [7]:
# A reguler expresion to check the last word in element name which is usally its type [street, road, etc...] 
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

# What we want to see
expected = ["Avenue", "Boulevard", "Commons", "Court", "Drive", "Lane", "Parkway", 
                         "Place", "Road", "Square", "Street", "Trail"]

# Mapping appriviations to expected.
mapping = {'Ave'  : 'Avenue',
           'Blvd' : 'Boulevard',
           'Dr'   : 'Drive',
           'Ln'   : 'Lane',
           'Pkwy' : 'Parkway',
           'Rd'   : 'Road',
           'Rd.'   : 'Road',
           'St'   : 'Street',
           'street' :"Street",
           'Ct'   : "Court",
           'Cir'  : "Circle",
           'Cr'   : "Court",
           'ave'  : 'Avenue',
           'Hwg'  : 'Highway',
           'Hwy'  : 'Highway',
           'Sq'   : "Square"}


In [8]:
# This function takes a dictunary of types and the street value. 
# If its ending not in expected add it to the dictonary.
def audit_street_type(street_types_dict, value):
    m = street_type_re.search(value)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types_dict[street_type].add(value)

# This function takes a filename, loop over nodes and ways then check if it has an address with street.
# If it is, audit its type.
def audit(filename):
    dataset = open(filename, "r")
    street_types_dict = defaultdict(set)
    for event, element in ET.iterparse(dataset, events=("start",)):
        if element.tag == "node" or element.tag == "way":
            for tag in element.iter("tag"):
                if tag.attrib['k'] == "addr:street":
                    audit_street_type(street_types_dict, tag.attrib['v'])

    return street_types_dict

types = audit(OSM_FILE)
pprint.pprint(dict(types))


{'2': set(['Canal Edge 2']),
 '3': set(['NY 3']),
 '434': set(['434']),
 '47': set(['St James str 47']),
 '500': set(['Quay 500']),
 '7806': set(['Spaanschemat River Rd,Cape Town, 7806']),
 'Aandblom': set(['Aandblom']),
 'Acre': set(['Pine Acre']),
 'Alexander': set(['Alexander']),
 'Avon': set(['Glen Avon']),
 'Avonduur': set(['Avonduur']),
 'Basin': set(['Victoria Basin']),
 'Bend': set(['Burgee Bend', 'The Bend']),
 'Bergzicht': set(['Bergzicht']),
 'Bird': set(['Bird']),
 u'Bleue': set([u'All\xe9e Bleue']),
 'Blom': set(['Blom']),
 'Bontheuvel': set(['Bontheuvel']),
 'Bosman': set(['Bosman']),
 'Buitengracht': set(['Buitengracht']),
 'Caledon': set(['Caledon']),
 'Camdebo': set(['Camdebo']),
 'Circle': set(['Cardita Circle',
                'Sacks Circle',
                'Vredenburg Circle',
                'Wesfleur Circle']),
 'Close': set(['Andre Close',
               'Annette Close',
               'Bolus Close',
               'Briony Close',
               'Daffodil Close'

The result above groups all elemetns based on thier endings. 

<hr />

#### 8.Change Abbreviations
Now, we will change old and wrong appriviations to the expected ones.

In [9]:
# This function takes the old name, the mapping of appriviations, and the types reguler expresion
# It try to find a regex in the old name
# If found, it update it then return the new name
# If not found, return old name as it is.
def update_name(name, mapping, regex):
    m = regex.search(name)
    if m:
        street_type = m.group()
        if street_type in mapping:
            name = re.sub(regex, mapping[street_type], name)

    return name

# For each type in types dictionary. Print the old name and the updated name. 
# Ex. Foam Rd => Foam Road
for street_type, ways in types.iteritems():
    for name in ways:
        better_name = update_name(name, mapping, street_type_re)
        print name, "=>", better_name


La Gratitude => La Gratitude
Molteno => Molteno
Oakland Hills => Oakland Hills
R310 => R310
Pienaar Rd => Pienaar Road
Foam Rd => Foam Road
Main Rd => Main Road
Prospect Rd => Prospect Road
Koetser => Koetser
Helderberg => Helderberg
Oosterland => Oosterland
Bird => Bird
Van Der Lingen => Van Der Lingen
Malherbe => Malherbe
Dorp => Dorp
Kiaat Singel => Kiaat Singel
Mahogany Singel => Mahogany Singel
Zandvliet Singel => Zandvliet Singel
Van der Stel => Van der Stel
Ryneveld => Ryneveld
Princess Path => Princess Path
Glen Avon => Glen Avon
Leeuwenhof => Leeuwenhof
Spaanschemat River Rd,Cape Town, 7806 => Spaanschemat River Rd,Cape Town, 7806
Main Road, St James => Main Road, St James
Mancadan => Mancadan
Louw => Louw
Minserie => Minserie
test stree => test stree
Avenue Normandy => Avenue Normandy
Welgemeend => Welgemeend
Canal Edge 2 => Canal Edge 2
Jannasch => Jannasch
Eikendal => Eikendal
Buitengracht => Buitengracht
Simfonielaan Wes => Simfonielaan Wes
Krige => Krige
Herte => Herte
Mo

The above are changed items after procceing them.

<hr/>

#### 9.Convert Data to JSON
Our dataset is in XML format. We are going to store data in MonogoDB, but before that we have to change dataset format from XML to JSON. 

To start doing that, we have to set the structure of the object, then start changing items based on it. 

In [12]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
address_regex = re.compile(r'^addr\:')
street_regex = re.compile(r'^street')

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]

'''
This method takes an element and check if it is a node or a way. If not skip it.
It map the XML element to an object where we can store it as JSON.
'''
def convert_element(element):
    node = {}
    if element.tag == "node" or element.tag == "way" :
        node['type'] = element.tag
        # address details of element
        address = {}
        
        # for each attribute in the element, parse it into object's variable
        for attribute in element.attrib:
            if attribute in CREATED:
                if 'created' not in node:
                    node['created'] = {}
                node['created'][attribute] = element.get(attribute)
            elif attribute in ['lat', 'lon']:
                continue
            else:
                node[attribute] = element.get(attribute)
                
        # store posstion cordinates if the element has lon and lat [GPS details]
        if 'lat' in element.attrib and 'lon' in element.attrib:
            node['pos'] = [float(element.get('lat')), float(element.get('lon'))]

        # for each sub element of the root one
        for sub in element:
            # parse second-level tags for ways and populate `node_refs`
            if sub.tag == 'nd':
                if 'node_refs' not in node:
                    node['node_refs'] = []
                if 'ref' in sub.attrib:
                    node['node_refs'].append(sub.get('ref'))

            # skip the sub if it has no k or v
            if sub.tag != 'tag' or 'k' not in sub.attrib or 'v' not in sub.attrib:
                continue
                
            key = sub.get('k')
            val = sub.get('v')

            # skip the key if it is not well prepared
            if problemchars.search(key):
                continue

            # if it is an address, store it in clean way
            elif address_regex.search(key):
                key = key.replace('addr:', '')
                address[key] = val

            # for others
            else:
                node[key] = val
                
        # clean address and store it in the node
        if len(address) > 0:
            node['address'] = {}
            street_full = None
            street_dict = {}
            street_format = ['prefix', 'name', 'type']
            # for each key in address
            for key in address:
                val = address[key]
                if street_regex.search(key):
                    if key == 'street':
                        street_full = val
                    elif 'street:' in key:
                        street_dict[key.replace('street:', '')] = val
                else:
                    node['address'][key] = val
            # assign street_full or fallback to compile street dict
            if street_full:
                node['address']['street'] = street_full
            elif len(street_dict) > 0:
                node['address']['street'] = ' '.join([street_dict[key] for key in street_format])
        return node
    else:
        return None
    

def convert_file(filename):
    output = "{0}.json".format(filename)
    data = []
    with codecs.open(output, "w") as fw:
        for event, element in ET.iterparse(filename):
            obj = convert_element(element)
            if obj:
                data.append(obj)
                fw.write(json.dumps(obj) + "\n")
    return data

data_objects = convert_file(OSM_FILE)
data_objects

[{'created': {'changeset': '19306159',
   'timestamp': '2013-12-06T13:30:06Z',
   'uid': '282726',
   'user': 'kaiD',
   'version': '4'},
  'highway': 'traffic_signals',
  'id': '18401303',
  'pos': [-33.9322555, 18.8587291],
  'type': 'node'},
 {'created': {'changeset': '45743616',
   'timestamp': '2017-02-02T11:04:54Z',
   'uid': '527535',
   'user': 'Constable',
   'version': '3'},
  'id': '18401307',
  'pos': [-33.9303126, 18.8739193],
  'type': 'node'},
 {'created': {'changeset': '174693',
   'timestamp': '2007-07-28T10:10:23Z',
   'uid': '4357',
   'user': 'Ray Booysen',
   'version': '1'},
  'id': '18401310',
  'pos': [-33.9306823, 18.8748669],
  'type': 'node'},
 {'created': {'changeset': '134137',
   'timestamp': '2006-10-17T23:38:04Z',
   'uid': '3560',
   'user': 'Firefishy',
   'version': '1'},
  'created_by': 'JOSM',
  'id': '18401311',
  'pos': [-33.9228417, 18.8546412],
  'type': 'node'},
 {'created': {'changeset': '169888',
   'timestamp': '2006-12-17T23:45:46Z',
   'ui

You can see the dataset converted to JSON format and saved on the local folder.

<hr/>

#### 10.Store Data in MongoDB
After cleaning data and store it in a clean structure, we are going to save it on local database using MongoDB. 

In [15]:
# start connection
client = MongoClient('localhost:27017')
db = client['map']
# add items one by one to DB
for item in data_objects:
    db.map.insert_one(item)

A long process ends with adding all items to the database.

<hr/>

#### 11.Statistical Overview
The last step is about showing statistical overview about the database.
<ul>
    <li>Size of the file</li>
    <li>Total elements</li>
    <li>Number of unique users</li>
    <li>Number of nodes and ways</li>
    <li>Number of chosen type of nodes, like cafes, shops etc.</li>
</ul>

#### Size of the file
What do you think, which file is bigger, the old or the new one?

In [16]:
# get the old file bytes and convert it to size
old_size = convert_bytes_to_size(os.path.getsize(OSM_FILE))

# get the new file bytes and convert it to size
new_size = convert_bytes_to_size(os.path.getsize(OSM_FILE + ".json"))


print ("The old file size is: {}.".format(old_size))
print ("The new file size is: {}.".format(new_size))

The old file size is: 283.0 MB.
The new file size is: 324.6 MB.


#### Total elements
Here is how we can count the number of elements in the database.

In [17]:
# total elements in DB
db.map.find().count()

1563030

#### Number of unique users
Now we are going to calculate the number of users who participated in the map.

In [18]:
# total number of users
unique_users = len(db.map.distinct('created.user'))
print ("Total users who participated in the map is: {} users.".format(unique_users))

Total users who participated in the map is: 1529 users.


#### Number of nodes and ways
We are going to count the number of nodes and ways in the database.

In [19]:
# count items where type = way
total_ways = db.map.find({'type':'way'}).count()

# count items where type = node
total_nodes = db.map.find({'type':'node'}).count()

print ("Total ways in the map is: {} ways.".format(total_ways))
print ("Total nodes in the map is: {} nodes.".format(total_nodes))


Total ways in the map is: 212127 ways.
Total nodes in the map is: 1350860 nodes.


#### Number of chosen type of nodes, like cafes, shops etc.
We are going to see togather what are the most popular nodes in selected area.

In [20]:
amenities = db.map.aggregate([{"$match":{"amenity":{"$exists":1}}}, {"$group":{"_id":"$amenity",
    "count":{"$sum":1}}}, {"$sort":{"count":-1}}, {"$limit":10}])

pprint.pprint(list(amenities))


[{u'_id': u'parking', u'count': 1719},
 {u'_id': u'restaurant', u'count': 556},
 {u'_id': u'school', u'count': 523},
 {u'_id': u'toilets', u'count': 441},
 {u'_id': u'drinking_water', u'count': 323},
 {u'_id': u'place_of_worship', u'count': 307},
 {u'_id': u'fuel', u'count': 302},
 {u'_id': u'fast_food', u'count': 252},
 {u'_id': u'waste_basket', u'count': 187},
 {u'_id': u'atm', u'count': 174}]


#### Top 5 religons 
We are going to investigate the **place_of_worship** to see top religons.

In [21]:
religions = db.map.aggregate([{"$match":{"amenity":{"$exists":1}, "amenity":"place_of_worship"}},
                 {"$group":{"_id":"$religion", "count":{"$sum":1}}},{"$sort":{"count":-1}}, {"$limit":5}])
list(religions)

[{u'_id': u'christian', u'count': 239},
 {u'_id': u'muslim', u'count': 35},
 {u'_id': None, u'count': 23},
 {u'_id': u'hindu', u'count': 5},
 {u'_id': u'jewish', u'count': 5}]

#### Top 5 food types
We are going to check best and top 5 food types [restaurant style].

In [22]:
places = db.map.aggregate([{"$match":{"amenity":{"$exists":1}, "amenity":"restaurant"}},
                           {"$group":{"_id":"$cuisine", "count":{"$sum":1}}},
                           {"$sort":{"count":-1}}, {"$limit":5}])
list(places)

[{u'_id': None, u'count': 311},
 {u'_id': u'regional', u'count': 42},
 {u'_id': u'italian', u'count': 24},
 {u'_id': u'pizza', u'count': 21},
 {u'_id': u'steak_house', u'count': 14}]

<hr/>

## 4.Conclusion
In conclusion, Open Street Map is an amazing project. It is going to help the community and improve many applications as it is an open source project. I liked the way how people participate in the project and help each other.

I suggest the site managers help people developing automated bots to correct the data via Google Maps API. This will allow us to have a copy from Google Map data, also Google Maps will get benefits from the community.


*Refs:*
<ul>
    <li>Wikipedia</li>
    <li>Udacity</li>
    <li>Stack Overflow</li>
</ul>